In [123]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from statistics import mean
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [23]:
df = pd.read_csv('SPX Index.csv', header=[0,1], index_col = 0)

In [24]:
#count number of attributes --> att_number
first_attribute = df.columns[0][1]
att_number = 1
att_name = []
for col in df.columns[1:]:
    if (col[1] != first_attribute):
        att_number += 1
        att_name.append(col[1])
    else:
        break
print(att_number)

21


In [4]:
#change level 0 row name from unnamed to company name
count = 2
company_list = []
for company in df.columns[::att_number]:
    company_list.append(company[0])
    for i in range(att_number):
        df.rename(columns={'Unnamed: {}_level_0'.format(count): company[0]}, inplace=True)
        count += 1

In [5]:
print(len(company_list))

505


In [6]:
df.index = pd.to_datetime(df.index)

In [7]:
te = df.copy()

In [58]:
df = te.copy()

In [59]:
for i in range(0, df.shape[1] + len(company_list), att_number+1): #account for adding DAILY_RETURN
    values = df[(df.columns[i][0], 'PX_LAST')].pct_change()
    df.insert(i, (df.columns[i][0],'DAILY_RETURN'), values)
att_number += 1 #DAILY_RETURN added to each company
df = df[1:] #drop first row

In [60]:
df

A UN Equity                                          \
Dates      DAILY_RETURN   PX_LAST PE_RATIO FREE_CASH_FLOW_YIELD   
2011-03-01     0.010862   29.9478  12.7400               5.7005   
2011-04-01    -0.009310   29.6690  12.6214               5.7541   
2011-05-01    -0.002171   29.6046  12.5940               5.7666   
2011-06-01     0.001932   29.6618  12.6183               5.7555   
2011-07-01     0.003375   29.7619  12.6609               5.7361   
...                 ...       ...      ...                  ...   
2021-03-29    -0.001751  125.4200  42.9639               2.8256   
2021-03-30    -0.004624  124.8400  42.7652               2.8388   
2021-03-31     0.018424  127.1400  43.5531               2.7874   
2021-01-04     0.004326  127.6900  43.7415               2.7754   
2021-02-04     0.000000  127.6900  43.7415               2.7754   

                                                                             \
Dates      PX_TO_BOOK_RATIO DVD_PAYOUT_RATIO NET_DEBT_TO_EBITDA PROF_MARGIN   
2011-03-01           3.2100              NaN                NaN         NaN   
2011-04-01           3.1801              NaN                NaN         NaN   
2011-05-01           3.1732              NaN                NaN         NaN   
2011-06-01           3.1794              NaN                NaN         NaN   
2011-07-01           3.1901              NaN                NaN         NaN   
...                     ...              ...                ...         ...   
2021-03-29           7.9628           19.125             1.0338     18.6047   
2021-03-30           7.9259           19.125             1.0338     18.6047   
2021-03-31           8.0720           19.125             1.0338     18.6047   
2021-01-04           8.1069           19.125             1.0338     18.6047   
2021-02-04           8.1069           19.125             1.0338     18.6047   

                                                            ... ZTS UN Equity  \
Dates      CURRENT_EV_TO_T12M_EBITDA RETURN_ON_INV_CAPITAL  ...   MOV_AVG_50D   
2011-03-01                   20.3225                   NaN  ...           NaN   
2011-04-01                   20.1460                   NaN  ...           NaN   
2011-05-01                   20.1052                   NaN  ...           NaN   
2011-06-01                   20.1414                   NaN  ...           NaN   
2011-07-01                   20.2048                   NaN  ...           NaN   
...                              ...                   ...  ...           ...   
2021-03-29                   30.3849               11.1135  ...      157.2252   
2021-03-30                   30.2492               11.1135  ...      157.1348   
2021-03-31                   30.7874               11.1135  ...      157.0330   
2021-01-04                   30.9161               11.1135  ...      156.9236   
2021-02-04                   30.9161               11.1135  ...      156.9236   

                                                                      \
Dates      MOV_AVG_100D MOV_AVG_200D RETURN_ON_ASSET PX_TO_CASH_FLOW   
2011-03-01          NaN          NaN             NaN             NaN   
2011-04-01          NaN          NaN             NaN             NaN   
2011-05-01          NaN          NaN             NaN             NaN   
2011-06-01          NaN          NaN             NaN             NaN   
2011-07-01          NaN          NaN             NaN             NaN   
...                 ...          ...             ...             ...   
2021-03-29     160.2180     156.7708         13.0238         35.4263   
2021-03-30     160.1570     156.8857         13.0238         35.1243   
2021-03-31     159.9883     157.0066         13.0238         35.2228   
2021-01-04     159.8582     157.1148         13.0238         35.3145   
2021-02-04     159.8582     157.1148         13.0238         35.3145   

                                                                        \
Dates      PX_TO_SALES_RATIO CAP_EXPEND_TO_SALE

In [61]:
att_name.insert(0, 'DAILY_RETURN')

In [62]:
print(len(att_name))

22


In [63]:
#check for first non-null value in each company, i = PX_LAST
dict_startrow = {}
for i in range(0,df.shape[1], att_number):
    dict_startrow[df.columns[i][0]] = df.iloc[:,i].first_valid_index()
    #print(df.columns[i][0], dict_startrow[df.columns[i][0]])

In [64]:
#stage1 cleaning, drop companies that enter index after 2013
#dataframe become df1
companies_todrop = []
for key, value in dict_startrow.items():
    if value >= pd.Timestamp(2013, 1, 1):
        companies_todrop.append(key)
        company_list.remove(key)
df1 = df.drop(companies_todrop, level=0, axis=1, inplace=False)
print(companies_todrop)

['AAL UW Equity', 'AEP UW Equity', 'ALLE UN Equity', 'AMCR UN Equity', 'AMD UW Equity', 'ANET UN Equity', 'APA UW Equity', 'CARR UN Equity', 'CDW UW Equity', 'CFG UN Equity', 'CSX UW Equity', 'CTLT UN Equity', 'CTVA UN Equity', 'CZR UW Equity', 'DOW UN Equity', 'DXC UN Equity', 'ETSY UW Equity', 'EXC UW Equity', 'FOX UW Equity', 'FOXA UW Equity', 'FTV UN Equity', 'GOOG UW Equity', 'HLT UN Equity', 'HPE UN Equity', 'HST UW Equity', 'HWM UN Equity', 'INFO UN Equity', 'IQV UN Equity', 'IR UN Equity', 'KEYS UN Equity', 'KHC UW Equity', 'LNT UW Equity', 'LW UN Equity', 'MAR UW Equity', 'NCLH UN Equity', 'NWL UW Equity', 'NWS UW Equity', 'NWSA UW Equity', 'ORCL UN Equity', 'OTIS UN Equity', 'PAYC UN Equity', 'PEP UW Equity', 'PFG UW Equity', 'PRGO UN Equity', 'PYPL UW Equity', 'QRVO UW Equity', 'REG UW Equity', 'SYF UN Equity', 'TER UW Equity', 'TMUS UW Equity', 'TWTR UN Equity', 'UA UN Equity', 'VIAC UW Equity', 'WBA UW Equity', 'WLTW UW Equity', 'WRK UN Equity', 'XEL UW Equity', 'ZTS UN Eq

In [65]:
print(len(companies_todrop))

58


In [70]:
#check number of null values
total_null_values = dict(df[df.columns].isnull().sum())
#for key,value in total_null_values.items():
    #print(key, ':', value)

In [71]:
#return percentage of null values in the dataframe
#remove attribute for all companies if percentage less than 70%
percent_null = {}
for attribute in range(0, att_number):
    null_temp = 0
    for i in range(attribute, df1.shape[1], att_number):
        null_temp += df1.iloc[:, i].isnull().sum()
    percent_null[df.columns[attribute][1]] = null_temp / (df.shape[0] * (df.shape[1] / att_number)) * 100
for key, value in percent_null.items():
    print(key, ' : ', value)

DAILY_RETURN  :  0.3420005551957065
PX_LAST  :  0.3406680855001388
PE_RATIO  :  2.600610715277135
FREE_CASH_FLOW_YIELD  :  1.4707504395299342
PX_TO_BOOK_RATIO  :  1.9799019154251873
DVD_PAYOUT_RATIO  :  1.450393263625428
NET_DEBT_TO_EBITDA  :  8.544165818451004
PROF_MARGIN  :  0.6536504117701489
CURRENT_EV_TO_T12M_EBITDA  :  8.928805403904876
RETURN_ON_INV_CAPITAL  :  1.0726381049319884
RSI_14D  :  0.5091514758952531
RSI_30D  :  0.534764504487832
MOV_AVG_50D  :  0.41861756269084854
MOV_AVG_100D  :  0.5078190061996853
MOV_AVG_200D  :  0.7097621911723883
RETURN_ON_ASSET  :  1.029480892014435
PX_TO_CASH_FLOW  :  1.607254557231424
PX_TO_SALES_RATIO  :  1.3583788285370593
CAP_EXPEND_TO_SALES  :  3.3190339594707137
CASH_CONVERSION_CYCLE  :  32.31083556953826
QUICK_RATIO  :  14.73274729342093
TOT_DEBT_TO_TOT_EQY  :  1.4329971314888497


In [72]:
df2 = df1.drop(['CASH_CONVERSION_CYCLE','QUICK_RATIO','CURRENT_EV_TO_T12M_EBITDA'], level=1, axis=1, inplace=False)
#dataframe becomes df2

In [75]:
print(len(company_list))

447


In [74]:
df1.shape

(2675, 9834)

In [73]:
df2.shape

(2675, 8493)

In [77]:
att_name.remove('CASH_CONVERSION_CYCLE')
att_name.remove('QUICK_RATIO')
att_name.remove('CURRENT_EV_TO_T12M_EBITDA')

In [78]:
print(len(att_name))

19


In [79]:
df2 = df2.groupby(pd.PeriodIndex(df2.index, freq='Q'), axis=0).transform(lambda x: x.fillna(x.mean()))

In [80]:
total_null_values = {}
count = 0
total_null_values = dict(df2[df2.columns].isnull().sum())
for key,value in total_null_values.items():
    if value != 0:
        count += value
        print(key, ' : ', value)

('ABBV UN Equity', 'DAILY_RETURN')  :  455
('ABBV UN Equity', 'PX_LAST')  :  455
('ABBV UN Equity', 'PE_RATIO')  :  455
('ABBV UN Equity', 'FREE_CASH_FLOW_YIELD')  :  521
('ABBV UN Equity', 'PX_TO_BOOK_RATIO')  :  455
('ABBV UN Equity', 'DVD_PAYOUT_RATIO')  :  260
('ABBV UN Equity', 'NET_DEBT_TO_EBITDA')  :  260
('ABBV UN Equity', 'RETURN_ON_INV_CAPITAL')  :  455
('ABBV UN Equity', 'RSI_14D')  :  455
('ABBV UN Equity', 'RSI_30D')  :  521
('ABBV UN Equity', 'MOV_AVG_50D')  :  521
('ABBV UN Equity', 'MOV_AVG_100D')  :  521
('ABBV UN Equity', 'MOV_AVG_200D')  :  521
('ABBV UN Equity', 'RETURN_ON_ASSET')  :  455
('ABBV UN Equity', 'PX_TO_CASH_FLOW')  :  455
('ABBV UN Equity', 'PX_TO_SALES_RATIO')  :  455
('ABBV UN Equity', 'TOT_DEBT_TO_TOT_EQY')  :  260
('ABMD UW Equity', 'PE_RATIO')  :  260
('ABMD UW Equity', 'NET_DEBT_TO_EBITDA')  :  260
('ADI UW Equity', 'DAILY_RETURN')  :  260
('ADI UW Equity', 'PX_LAST')  :  260
('ADI UW Equity', 'PE_RATIO')  :  260
('ADI UW Equity', 'FREE_CASH_FLOW_Y

In [81]:
df2.shape

(2675, 8493)

In [82]:
df2.drop('BRK/B UN Equity', level=0, axis=1, inplace=True)

In [83]:
df2.shape

(2675, 8474)

In [84]:
print(len(company_list))

447


In [85]:
company_list.remove('BRK/B UN Equity')

In [86]:
print(len(company_list))

446


In [87]:
#return percentage of null values in the dataframe
#remove attribute for all companies if percentage less than 70%
percent_null2 = {}
for attribute in range(0, att_number):
    null_temp = 0
    for i in range(attribute, df2.shape[1], att_number):
        null_temp += df2.iloc[:, i].isnull().sum()
    percent_null2[df.columns[attribute][1]] = null_temp / (df2.shape[0] * (df2.shape[1] / att_number)) * 100
for key, value in percent_null2.items():
    print(key, ' : ', value)

DAILY_RETURN  :  1.1955029016739493
PX_LAST  :  1.5308221519811012
PE_RATIO  :  0.9362734609878706
FREE_CASH_FLOW_YIELD  :  0.9928555515606837
PX_TO_BOOK_RATIO  :  0.909874955609131
DVD_PAYOUT_RATIO  :  2.2715331558433824
NET_DEBT_TO_EBITDA  :  1.3852421590836401
PROF_MARGIN  :  2.068303485758527
CURRENT_EV_TO_T12M_EBITDA  :  1.6884367576247523
RETURN_ON_INV_CAPITAL  :  2.3469435921642674
RSI_14D  :  1.1825948089703744
RSI_30D  :  1.8852609080221194
MOV_AVG_50D  :  0.8601836513667976
MOV_AVG_100D  :  1.0943733332745131
MOV_AVG_200D  :  0.7524544566226765
RETURN_ON_ASSET  :  1.3846598391120502
PX_TO_CASH_FLOW  :  1.7208525693765866
PX_TO_SALES_RATIO  :  1.6316605603947425
CAP_EXPEND_TO_SALES  :  1.8028626320421564
CASH_CONVERSION_CYCLE  :  1.2518908855895658
QUICK_RATIO  :  1.9682415039736718
TOT_DEBT_TO_TOT_EQY  :  1.7145441030176969


In [88]:
print(len(company_list))

446


In [89]:
print(att_name)
print(len(att_name))

['DAILY_RETURN', 'DAILY_RETURN', 'PE_RATIO', 'FREE_CASH_FLOW_YIELD', 'PX_TO_BOOK_RATIO', 'DVD_PAYOUT_RATIO', 'NET_DEBT_TO_EBITDA', 'PROF_MARGIN', 'RETURN_ON_INV_CAPITAL', 'RSI_14D', 'RSI_30D', 'MOV_AVG_50D', 'MOV_AVG_100D', 'MOV_AVG_200D', 'RETURN_ON_ASSET', 'PX_TO_CASH_FLOW', 'PX_TO_SALES_RATIO', 'CAP_EXPEND_TO_SALES', 'TOT_DEBT_TO_TOT_EQY']
19


In [91]:
df2.shape

(2675, 8474)

In [94]:
train_df = df2.iloc[:math.floor(df2.shape[0]/2),]
test_df = df2.iloc[math.ceil(df2.shape[0]/2):,]

In [95]:
daily_return = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='DAILY_RETURN'])
daily_return = daily_return.transpose()
daily_return = daily_return.droplevel(1)

In [97]:
pe_ratio = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='PE_RATIO'])
pe_ratio = pe_ratio.transpose()
pe_ratio = pe_ratio.droplevel(1)

In [99]:
free_cash_flow_yield = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='FREE_CASH_FLOW_YIELD'])
free_cash_flow_yield = free_cash_flow_yield.transpose()
free_cash_flow_yield = free_cash_flow_yield.droplevel(1)

In [100]:
px_to_book_ratio = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='PX_TO_BOOK_RATIO'])
px_to_book_ratio = px_to_book_ratio.transpose()
px_to_book_ratio = px_to_book_ratio.droplevel(1)

In [101]:
dvd_payout_ratio = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='DVD_PAYOUT_RATIO'])
dvd_payout_ratio = dvd_payout_ratio.transpose()
dvd_payout_ratio = dvd_payout_ratio.droplevel(1)

In [102]:
net_debt_to_ebitda = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='NET_DEBT_TO_EBITDA'])
net_debt_to_ebitda = net_debt_to_ebitda.transpose()
net_debt_to_ebitda = net_debt_to_ebitda.droplevel(1)

In [103]:
prof_margin = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='PROF_MARGIN'])
prof_margin = prof_margin.transpose()
prof_margin = prof_margin.droplevel(1)

In [104]:
return_on_inv_capital = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='RETURN_ON_INV_CAPITAL'])
return_on_inv_capital = return_on_inv_capital.transpose()
return_on_inv_capital = return_on_inv_capital.droplevel(1)

In [105]:
rsi_14d = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='RSI_14D'])
rsi_14d = rsi_14d.transpose()
rsi_14d = rsi_14d.droplevel(1)

In [106]:
rsi_30d = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='RSI_30D'])
rsi_30d = rsi_30d.transpose()
rsi_30d = rsi_30d.droplevel(1)

In [107]:
mov_avg_50d = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='MOV_AVG_50D'])
mov_avg_50d = mov_avg_50d.transpose()
mov_avg_50d = mov_avg_50d.droplevel(1)

In [108]:
mov_avg_100d = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='MOV_AVG_100D'])
mov_avg_100d = mov_avg_100d.transpose()
mov_avg_100d = mov_avg_100d.droplevel(1)

In [109]:
mov_avg_200d = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='MOV_AVG_200D'])
mov_avg_200d = mov_avg_200d.transpose()
mov_avg_200d = mov_avg_200d.droplevel(1)

In [110]:
return_on_asset = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='RETURN_ON_ASSET'])
return_on_asset = return_on_asset.transpose()
return_on_asset = return_on_asset.droplevel(1)

In [111]:
px_to_cash_flow = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='PX_TO_CASH_FLOW'])
px_to_cash_flow = px_to_cash_flow.transpose()
px_to_cash_flow = px_to_cash_flow.droplevel(1)

In [112]:
px_to_sales_ratio = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='PX_TO_SALES_RATIO'])
px_to_sales_ratio = px_to_sales_ratio.transpose()
px_to_sales_ratio = px_to_sales_ratio.droplevel(1)

In [113]:
cap_expend_to_sales = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='CAP_EXPEND_TO_SALES'])
cap_expend_to_sales = cap_expend_to_sales.transpose()
cap_expend_to_sales = cap_expend_to_sales.droplevel(1)

In [114]:
tot_debt_to_tot_eqy = pd.DataFrame(train_df.iloc[:, train_df.columns.get_level_values(1)=='TOT_DEBT_TO_TOT_EQY'])
tot_debt_to_tot_eqy = tot_debt_to_tot_eqy.transpose()
tot_debt_to_tot_eqy = tot_debt_to_tot_eqy.droplevel(1)

In [115]:
model = LinearRegression()

In [128]:
def p_value():
    ttest,pval = stats.ttest_rel(regr_data.iloc[0, :], regr_data.iloc[1, :])
    return pval

In [166]:
pe_ratio_alpha = []
pe_ratio_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[pe_ratio.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    pe_ratio_alpha.append(model.intercept_)
    pe_ratio_beta.append(model.coef_[0])
print(mean(pe_ratio_alpha))
print(mean(pe_ratio_beta))

0.000503211748167696
1.0828668401107538e-06


In [162]:
free_cash_flow_yield_alpha = []
free_cash_flow_yield_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[free_cash_flow_yield.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    free_cash_flow_yield_alpha.append(model.intercept_)
    free_cash_flow_yield_beta.append(model.coef_[0])
print(mean(free_cash_flow_yield_alpha))
print(mean(free_cash_flow_yield_beta))

0.00056790744270942
-2.62415919507881e-06


In [163]:
px_to_book_ratio_alpha = []
px_to_book_ratio_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[px_to_book_ratio.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    px_to_book_ratio_alpha.append(model.intercept_)
    px_to_book_ratio_beta.append(model.coef_[0])
print(mean(px_to_book_ratio_alpha))
print(mean(px_to_book_ratio_beta))

0.0005189464435844311
4.312798598702772e-06


In [164]:
dvd_payout_ratio_alpha = []
dvd_payout_ratio_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[dvd_payout_ratio.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    dvd_payout_ratio_alpha.append(model.intercept_)
    dvd_payout_ratio_beta.append(model.coef_[0])
print(mean(dvd_payout_ratio_alpha))
print(mean(dvd_payout_ratio_beta))

0.000554278528317143
-2.2030624856326845e-07


In [167]:
net_debt_to_ebitda_alpha = []
net_debt_to_ebitda_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[net_debt_to_ebitda.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    net_debt_to_ebitda_alpha.append(model.intercept_)
    net_debt_to_ebitda_beta.append(model.coef_[0])
print(mean(net_debt_to_ebitda_alpha))
print(mean(net_debt_to_ebitda_beta))

0.0005763392137405066
-1.713254849045817e-05


In [168]:
prof_margin_alpha = []
prof_margin_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[prof_margin.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    prof_margin_alpha.append(model.intercept_)
    prof_margin_beta.append(model.coef_[0])
print(mean(prof_margin_alpha))
print(mean(prof_margin_beta))

0.0005582097650499273
-1.3262474933687633e-06


In [173]:
return_on_inv_capital_alpha = []
return_on_inv_capital_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[return_on_inv_capital.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    return_on_inv_capital_alpha.append(model.intercept_)
    return_on_inv_capital_beta.append(model.coef_[0])
print(mean(return_on_inv_capital_alpha))
print(mean(return_on_inv_capital_beta))

0.0004862915098656588
4.810890702348492e-06


In [174]:
rsi_14d_alpha = []
rsi_14d_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[rsi_14d.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    rsi_14d_alpha.append(model.intercept_)
    rsi_14d_beta.append(model.coef_[0])
print(mean(rsi_14d_alpha))
print(mean(rsi_14d_beta))

-0.02220811586397165
0.0004375746977050012


In [175]:
rsi_30d_alpha = []
rsi_30d_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[rsi_30d.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    rsi_30d_alpha.append(model.intercept_)
    rsi_30d_beta.append(model.coef_[0])
print(mean(rsi_30d_alpha))
print(mean(rsi_30d_beta))

-0.022791927827497917
0.000447816959120088


In [176]:
mov_avg_50d_alpha = []
mov_avg_50d_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[mov_avg_50d.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    mov_avg_50d_alpha.append(model.intercept_)
    mov_avg_50d_beta.append(model.coef_[0])
print(mean(mov_avg_50d_alpha))
print(mean(mov_avg_50d_beta))

0.000569057823688267
-5.552737144621137e-07


In [177]:
mov_avg_100d_alpha = []
mov_avg_100d_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[mov_avg_100d.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    mov_avg_100d_alpha.append(model.intercept_)
    mov_avg_100d_beta.append(model.coef_[0])
print(mean(mov_avg_100d_alpha))
print(mean(mov_avg_100d_beta))

0.0005673726027980963
-5.488811772393479e-07


In [178]:
mov_avg_200d_alpha = []
mov_avg_200d_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[mov_avg_200d.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    mov_avg_200d_alpha.append(model.intercept_)
    mov_avg_200d_beta.append(model.coef_[0])
print(mean(mov_avg_200d_alpha))
print(mean(mov_avg_200d_beta))

0.0005693162494223291
-5.609177614925016e-07


In [179]:
return_on_asset_alpha = []
return_on_asset_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[return_on_asset.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    return_on_asset_alpha.append(model.intercept_)
    return_on_asset_beta.append(model.coef_[0])
print(mean(return_on_asset_alpha))
print(mean(return_on_asset_beta))

0.0005311762518495181
1.7284820855529712e-06


In [180]:
px_to_cash_flow_alpha = []
px_to_cash_flow_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[px_to_cash_flow.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    px_to_cash_flow_alpha.append(model.intercept_)
    px_to_cash_flow_beta.append(model.coef_[0])
print(mean(px_to_cash_flow_alpha))
print(mean(px_to_cash_flow_beta))

0.000474329250154573
3.974680978963359e-06


In [181]:
px_to_sales_ratio_alpha = []
px_to_sales_ratio_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[px_to_sales_ratio.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    px_to_sales_ratio_alpha.append(model.intercept_)
    px_to_sales_ratio_beta.append(model.coef_[0])
print(mean(px_to_sales_ratio_alpha))
print(mean(px_to_sales_ratio_beta))

0.0003448707925660071
6.668163479549944e-05


In [182]:
cap_expend_to_sales_alpha = []
cap_expend_to_sales_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[cap_expend_to_sales.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    cap_expend_to_sales_alpha.append(model.intercept_)
    cap_expend_to_sales_beta.append(model.coef_[0])
print(mean(cap_expend_to_sales_alpha))
print(mean(cap_expend_to_sales_beta))

0.0005744110177510851
-2.174474340609308e-06


In [184]:
tot_debt_to_tot_eqy_alpha = []
tot_debt_to_tot_eqy_beta = []
for i in range(train_df.shape[0]):
    regr_data = pd.DataFrame(data=[tot_debt_to_tot_eqy.iloc[:, i], daily_return.iloc[:, i]])
    regr_data.dropna(axis=1, inplace=True)
    model.fit(regr_data.iloc[0, :].values.reshape(-1,1), regr_data.iloc[1, :])
    tot_debt_to_tot_eqy_alpha.append(model.intercept_)
    tot_debt_to_tot_eqy_beta.append(model.coef_[0])
print(mean(tot_debt_to_tot_eqy_alpha))
print(mean(tot_debt_to_tot_eqy_beta))

0.0005412643424732849
-1.448661697196833e-09
